In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
import os
from tqdm import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

2024-06-01 02:30:03.050025: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 02:30:03.144769: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
gpus = tf.config.experimental.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
print('\n\n Running on multiple GPUs ', [gpu.name for gpu in gpus])

2024-06-01 02:30:14.475835: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 02:30:16.111666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14825 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:3b:00.0, compute capability: 7.5
2024-06-01 02:30:16.112763: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14825 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:5e:00.0, compute capability: 7.5
2024-06-01 02:30:16.113764: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:local



 Running on multiple GPUs  ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3']


In [3]:
with strategy.scope():
    w, h = 19, 19
    base_path = os.path.join('dataset3', '*/*.npz')
    
    file_list = glob(base_path)
    
    x_data, y_data = [], []
    for file_path in tqdm(file_list):
        data = np.load(file_path)
        x_data.extend(data['inputs'])
        y_data.extend(data['outputs'])
    
    x_data = np.array(x_data, np.float4).reshape((-1, h, w, 1))
    y_data = np.array(y_data, np.float4).reshape((-1, h * w))
    
    x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2020)
    
    del x_data, y_data
    
    print(x_train.shape, y_train.shape)
    print(x_val.shape, y_val.shape)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 24938/24938 [00:53<00:00, 468.38it/s]


AttributeError: module 'numpy' has no attribute 'float4'

In [4]:
with strategy.scope():
    model = models.Sequential([
        layers.Conv2D(32, 7, activation='relu', padding='same', input_shape=(19, 19, 1)),
        layers.Conv2D(64, 7, activation='relu', padding='same',),
        layers.Conv2D(128, 7, activation='relu', padding='same'),
#        layers.Conv2D(256, 7, activation='relu', padding='same'),
#        layers.Conv2D(128, 7, activation='relu', padding='same'),
        layers.Conv2D(64, 7, activation='relu', padding='same'),
        layers.Conv2D(32, 7, activation='relu', padding='same'),
        layers.Conv2D(1, 1, activation=None, padding='same'),  
        layers.Reshape((h * w,)),
        layers.Activation('sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['acc']
    )
    
    model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 19, 19, 32)        1600      
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 64)        100416    
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 19, 128)       401536    
                                                                 
 conv2d_3 (Conv2D)           (None, 19, 19, 64)        401472    
                                                                 
 conv2d_4 (Conv2D)           (None, 19, 19, 32)        100384    
                                                                 
 conv2d_5 (Conv2D)           (None, 19, 19, 1)         33        
                                                                 
 reshape (Reshape)           (None, 361)               0

In [5]:
with strategy.scope():
    start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
    os.makedirs('models', exist_ok=True)
    
    model.fit(
        x=x_train,
        y=y_train,
        batch_size=256,
        epochs=10,
        callbacks=[
            ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
            ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
        ],
        validation_data=(x_val, y_val),
        use_multiprocessing=True,
        workers=16
    )

2024-06-01 02:25:59.449718: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 12.85GiB (rounded to 13800683520)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-06-01 02:25:59.449782: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2024-06-01 02:25:59.449808: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 21, Chunks in use: 21. 5.2KiB allocated for chunks. 5.2KiB in use in bin. 968B client-requested in use in bin.
2024-06-01 02:25:59.449824: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 512B allocated for chunks. 512B in use in bin. 512B client-requested in use in bin.
2024-06-01 02:25:59.449840: I tensorflow/tsl/framework/bfc_allo

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.